In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re

# from sklearn.linear_model import LogisticRegression
# from sklearn.naive_bayes import GaussianNB
# from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.ensemble import GradientBoostingClassifier
# from sklearn import svm
from sklearn. neighbors import LocalOutlierFactor
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, f1_score, roc_auc_score
# from sklearn.feature_selection import RFE
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import MinMaxScaler, StandardScaler, RobustScaler

from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis as LDA

# from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import PolynomialFeatures

from scipy.stats import expon, reciprocal
from xgboost import XGBClassifier, XGBRegressor
#from lightgbm import LGBMClassifier
from missingpy import MissForest

import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv('./data/galaxy_final.csv', index_col=0)
X = df.drop('sold', axis=1)
y = df.sold
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=11,
                                                       stratify=y, shuffle=True)

# XGBoost Tuning
- https://www.kaggle.com/lifesailor/xgboost
- https://brunch.co.kr/@snobberys/137
- https://xzz201920.medium.com/xgbosst-booster-gbtree-v-s-dart-v-s-gblinear-82d8fcbb07d2

General Parameter
- booster: tree 기반 모델 / 선형 모델
    - https://xzz201920.medium.com/xgbosst-booster-gbtree-v-s-dart-v-s-gblinear-82d8fcbb07d2
    - skip_drop(default = 0, range [0, 1]) is the probability of skipping dropout. It has a higher priority than other DART parameters.
        - If skip_drop = 1, the dropout procedure would be skipped and dart is the same as gbtree.
    - If skip_drop≠0, rate_drop (default = 0, range [0, 1]) will drop a fraction of the trees before the model update in every iteration.
        - dropout makes dart between gbtree and random forest: “If no tree is dropped, dart is the same as (gbtree); if all the trees are dropped, dart is no different than random forest.”
- silent: 메세지 조절
- nthread: 병렬 처리 조절

Boost Parameter
- eta: Learning rate(일반적으로 0.01 - 0.2)
- min_child_weight: min_child_weight를 기준으로 추가 분기 결정(크면 Underfitting)
- max_depth: Tree 깊이 수
- max_leaf_node: 하나의 트리에서 node 개수
- gamma: split 하기 위한 최소의 loss 감소 정의
- subsample: 데이터 중 샘플링(0.5 - 1)
- colsample_bytree: column 중 sampling(0.5 - 1)
- colsample_bylevel: 각 level마다 샘플링 비율
- lambda: L2 nrom
- alpha: L1 norm
- scale_pos_weight: positive, negative weight 지정
- 기타 등

Learning Task Parameter
- object: 목적함수 종류
    - binary:logistic(이진 분류)
    - multi:softmax(다중 분류)
    - multi-softprob(다중 확률)
- eval_metric: 평가 지표
    - rmse – root mean square error
    - mae – mean absolute error
    - logloss – negative log-likelihood
    - error – Binary classification error rate (0.5 threshold)
    - merror – Multiclass classification error rate
    - mlogloss – Multiclass logloss
    - auc: Area under the curve
seed

### Overview
- high learning rate(0.05 - 0.3)를 선택하고 이 학습률에 맞는 tree 개수를 선정한다.
- tree-specific parameter를 수정한다.
- max_depth, min_child_weight, gamma, subsample, colsample_bytree
- regularization parameter를 수정한다.
- 학습률을 낮추고 다시 반복한다.

### Initialization
초기값은 다음과 같이 선정한다.
- max_depth = 5: 보통 4-6 를 시작점으로 한다.
- min_child_weight = 1 : 향후에 튜닝할 것이다.
- gamma = 0 : 0.1 - 0.2로 시작해도 된다. 그런데 어짜피 튜닝할 것이다.
- subsample, colsample_bytree = 0.8 : 보통 0.5 - 0.9로 시작한다.
- scale_pos_weight = 1: Because of high class imbalance.


### 1-1. Scaler + PolynomialFeatures

In [4]:
pipe1_1 = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid1_1 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[1, 2, 3]
              }
             ] 
grid1_1 = GridSearchCV(pipe1_1, param_grid1_1, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_1.fit(X_train, y_train)
print(grid1_1.best_params_)
print(grid1_1.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 12.2min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### 1-2. Scaler Only

In [6]:
pipe1_2 = Pipeline([
                ('scale', MinMaxScaler()),
#                 ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid1_2 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
#                'poly':[PolynomialFeatures()],
#                'poly__degree':[1, 2, 3]
              }
             ] 
grid1_2 = GridSearchCV(pipe, param_grid1_2, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_2.fit(X_train, y_train)
print(grid1_2.best_params_)
print(grid1_2.best_score_)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.8min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### 1-3. Scale -> Poly -> ReduceDim

- PCA & LDA
    - https://huidea.tistory.com/126
- LDA
    - https://yamalab.tistory.com/41
    - http://www.datamarket.kr/xe/board_oFxn34/26649
- TSNE
    - https://agiantmind.tistory.com/215
    - https://lovit.github.io/nlp/representation/2018/09/28/tsne/

In [12]:
pipe1_3 = Pipeline([
                ('scale', MinMaxScaler()),
                 ('poly', PolynomialFeatures()),
                ('reduce_dims', PCA()),
                ('classifier', XGBClassifier())
                ])

param_grid1_3 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':[5],
               'classifier__min_child_weight':[1],
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[MinMaxScaler(), StandardScaler(), RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree': [1, 2, 3],
               'reduce_dims' : [PCA(), LDA(), TSNE()],
               'reduce_dims__n_components' : [5, 7, 9, 11]
              }
             ] 
grid1_3 = GridSearchCV(pipe1_3, param_grid1_3, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid1_3.fit(X_train, y_train)
print(grid1_3.best_params_)
print(grid1_3.best_score_)

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 280 tasks      | elapsed: 39.7min
[Parallel(n_jobs=-1)]: Done 540 out of 540 | elapsed: 64.3min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=5, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 5, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

### Scaler + Poly > Scaler Only > Scaler + Poly + DimReduction
- 0.805 > 0.797 > 0.789

### max_depth, min_child_weight

In [7]:
#max_depth와 min_child_weight를 튜닝한다.

pipe = Pipeline([
                ('scale', MinMaxScaler()),
                ('poly', PolynomialFeatures()),
                ('classifier', XGBClassifier())
                ])

param_grid2 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth':range(3,10,3),
               'classifier__min_child_weight':range(1,6,2),
               'classifier__gamma':[0],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid2 = GridSearchCV(pipe, param_grid2, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid2.fit(X_train, y_train)
print(grid2.best_params_)
print(grid2.best_score_)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed: 15.9min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs': -1,

In [18]:
# Gamma를 튜닝한다.
param_grid3 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[i/10.0 for i in range(0,10)],
               'classifier__subsample':[0.8],
               'classifier__colsample_bytree':[0.8],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid3 = GridSearchCV(pipe, param_grid3, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid3.fit(X_train, y_train)
print(grid3.best_params_)
print(grid3.best_score_)

Fitting 5 folds for each of 10 candidates, totalling 50 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.4min
[Parallel(n_jobs=-1)]: Done  50 out of  50 | elapsed: 11.9min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.8, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.8, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.8, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

In [20]:
# subsample and colsample_bytree를 튜닝한다.
param_grid4 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[i/10.0 for i in range(6,10)],
               'classifier__colsample_bytree':[i/10.0 for i in range(6,10)],
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid4 = GridSearchCV(pipe, param_grid4, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid4.fit(X_train, y_train)
print(grid4.best_params_)
print(grid4.best_score_)

Fitting 5 folds for each of 16 candidates, totalling 80 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done  80 out of  80 | elapsed: 19.2min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

In [21]:
# n_estimators 튜닝

param_grid5 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [0.15],
              'classifier__skip_drop': [0.33],
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[50, 100, 300, 500, 750, 1000],
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid5 = GridSearchCV(pipe, param_grid5, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid5.fit(X_train, y_train)
print(grid5.best_params_)
print(grid5.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  2.8min finished


{'classifier': XGBClassifier(base_score=None, booster='dart', colsample_bylevel=None,
              colsample_bynode=None, colsample_bytree=0.9, eval_metric='error',
              gamma=0.0, gpu_id=None, importance_type='gain',
              interaction_constraints=None, learning_rate=0.1,
              max_delta_step=None, max_depth=3, min_child_weight=1, missing=nan,
              monotone_constraints=None, n_estimators=1000, n_jobs=-1,
              nthread=-1, num_parallel_tree=None, objective='binary:logistic',
              random_state=None, rate_drop=0.15, reg_alpha=None,
              reg_lambda=None, scale_pos_weight=1, seed=2021, skip_drop=0.33,
              subsample=0.7, tree_method=None, ...), 'classifier__booster': 'dart', 'classifier__colsample_bytree': 0.9, 'classifier__eval_metric': 'error', 'classifier__gamma': 0.0, 'classifier__learning_rate': 0.1, 'classifier__max_depth': 3, 'classifier__min_child_weight': 1, 'classifier__n_estimators': 1000, 'classifier__n_jobs':

In [ ]:
# rate_drop, skip_drop 튜닝
param_grid6 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': np.arange(0.1, 0.55, 0.05),
              'classifier__skip_drop': np.arange(0.1, 0.55, 0.05),
             'classifier__learning_rate':[0.1],
             'classifier__n_estimators':[grid5.best_params_['classifier__n_estimators']], # 1000
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ] 
grid6 = GridSearchCV(pipe, param_grid6, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid6.fit(X_train, y_train)
print(grid6.best_params_)
print(grid6.best_score_)

Fitting 5 folds for each of 81 candidates, totalling 405 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 10.0min


In [ ]:
# learning rate

param_grid7 = [              
              {'classifier': [XGBClassifier()],
              'classifier__booster': ['dart'],
              'classifier__rate_drop': [grid6.best_params_['classifier__rate_drop']],
              'classifier__skip_drop': [grid6.best_params_['classifier__skip_drop']],
             'classifier__learning_rate': [0.01, 0.03, 0.05, 0.1],
             'classifier__n_estimators':[grid5.best_params_['classifier__n_estimators']], # 1000
               'classifier__max_depth': [grid2.best_params_['classifier__max_depth']], # 3
               'classifier__min_child_weight': [grid2.best_params_['classifier__min_child_weight']], # 1
               'classifier__gamma':[grid3.best_params_['classifier__gamma']], # 0.0
               'classifier__subsample':[grid4.best_params_['classifier__subsample']], # 0.7
               'classifier__colsample_bytree':[grid4.best_params_['classifier__colsample_bytree']], # 0.9
               'classifier__objective':['binary:logistic'],
               'classifier__nthread':[-1],
               'classifier__scale_pos_weight':[1],
               'classifier__seed':[2021],
               'classifier__eval_metric':['error'],
               'classifier__n_jobs':[-1],
               'scale':[RobustScaler()],
               'poly':[PolynomialFeatures()],
               'poly__degree':[3]
              }
             ]
grid7 = GridSearchCV(pipe, param_grid7, scoring = 'accuracy',
                    cv=StratifiedKFold(n_splits=5),
                    verbose=1, n_jobs=-1)
grid7.fit(X_train, y_train)
print(grid7.best_params_)
print(grid7.best_score_)

In [ ]:
# fit - early stopping rounds
xgb_best = grid7.best_estimator_

for esr in np.arange(0.1, 0.55, 0.05):
    xgb_best.fit(X_train, y_train, early_stopping_rounds=50, eval_metric="accuracy",
                eval_set=[(X_train, y_train), (X_test, y_test)])

# Local Outlier Factor

In [ ]:
test_neighbors = np.linspace(1, 100, num=20).astype(int)
test_contams = np.linspace(0.01, 0.2, num=20)
best_params, best_acc, X2, y2 = 0, 0, 0, 0

def tune_lof_by_model(model, df, scaler=None, poly=None, dim_reduction=None):
    best_params, best_acc, X2, y2 = 0, 0, 0, 0
    for i, tn in enumerate(test_neighbors):
        print(i, end='/')
        for j, tc in enumerate(test_contams):
            
            clf = LocalOutlierFactor(n_neighbors=tn, contamination=tc)
            y_pred = clf.fit_predict(df.drop('sold', axis=1))
            lof_outlier_idx = pd.Series(y_pred)[pd.Series(y_pred)==-1].index
            df_lof2 = df.drop(lof_outlier_idx)
            
            X2 = df_lof2.drop('sold', axis=1)
            y2 = df_lof2.sold
            X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2,
                                                                    test_size=0.2,
                                                                    shuffle=True,
                                                                    stratify=y2,
                                                                    random_state=11)
            
            if scaler:
                X2_train = scaler.fit_transform(X2_train)
                X2_test = scaler.transform(X2_test)
                
            if poly:
                X2_train = poly.fit_transform(X2_train)
                X2_test = poly.transform(X2_test)
            
            if dim_reduction:
                X2_train = dim_reduction.fit_transform(X2_train)
                X2_test = dim_reduction.transform(X2_test)
            
            mod = model
            mod.fit(X2_train, y2_train)
            mod_acc = accuracy_score(y2_test, mod.predict(X2_test))
            if best_acc < mod_acc:
                best_acc = mod_acc
                best_params = (tn, tc)
                X2 = X2
                y2 = y2
    return best_params, best_acc, X2, y2
        #print(accuracy_score(y2_test, lr.predict(X2_test)))
        #print(test_ensemble(X2_train, y2_train, X2_test, y2_test))

In [ ]:
lgbm_best = LGBMClassifier(application='binary', boosting_type='dart',
               categorical_feature=[0, 1, 3, 4, 5, 6, 8, 9, 10, 11, 12],
               class_weight=None, colsample_bytree=0.8, importance_type='split',
               learning_rate=0.03, max_depth=9, metric='binary_logloss',
               min_child_samples=16, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=16, objective='binary',
               random_state=None, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=0.5, subsample_for_bin=200000, subsample_freq=0,
                          drop_rate=0.4, skip_drop=0.4, num_iterations=1000)

lgbm_scaler = MinMaxScaler()
lgbm_poly = PolynomialFeatures(degree=3)
lgbm_lof_tune = tune_lof_by_model(lgbm_best, df,
                                  scaler=lgbm_scaler,
                                 poly=lgbm_poly)
lgbm_lof_tune[:2]